In [36]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs
from dateutil import parser,tz

In [37]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
url = f"https://www.metrotvnews.com/index/2023-12-01/0"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
body= soup.find('div',{"class":"item-list pt-20"})
articles = body.findAll('div', {"class": "text"})
links = []
for article in articles:
    link = article.find('a')['href']
    full_link = f"https://www.metrotvnews.com{link}"
    links.append(full_link)
    
print(links)

['https://www.metrotvnews.com/read/kewCaRZl-kpu-hapus-debat-khusus-cawapres', 'https://www.metrotvnews.com/read/bmRCepjy-saka-disebut-butuh-istirahat', 'https://www.metrotvnews.com/read/bzGCzLAX-pelatih-prancis-anggap-final-piala-dunia-u-17-seperti-laga-biasa', 'https://www.metrotvnews.com/read/bw6CoP4y-striker-prancis-minta-dukungan-lebih-besar-di-final-piala-dunia-u-17', 'https://www.metrotvnews.com/read/bD2C1qPa-kemenpora-dorong-pencak-silat-masuk-olimpiade', 'https://www.metrotvnews.com/read/NLMC2x69-jubir-amin-sebut-contract-farming-upayakan-hak-milik-lahan-petani', 'https://www.metrotvnews.com/read/b2lCVzjE-firli-singgung-ham-dan-serangan-balik-koruptor-usai-diperiksa-sebagai-tersangka', 'https://www.metrotvnews.com/read/NrWC5Pvd-format-baru-debat-cawapres-kini-bakal-didampingi-capres', 'https://www.metrotvnews.com/read/KRXC5VvV-sinergitas-jadi-kunci-sukses-piala-dunia-u-17', 'https://www.metrotvnews.com/read/NxGCzR4o-firli-bahuri-saling-pandang-dengan-alex-tirta-saat-diperiksa',

In [38]:
def scrape_links(date, page_number):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.metrotvnews.com/index/{date}/{page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body= soup.find('div',{"class":"item-list pt-20"})
    articles =  body.find_all('div', {"class": "text"})
    
    links = []
    for article in articles:
        link = article.find('a')['href']
        full_link = f"https://www.metrotvnews.com{link}"
        links.append(full_link)
        
    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    return links

In [39]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 0
    page_links = []
    # Adjust the date format here
    # current_date = datetime.strptime(date, "%Y-%m-%d")
    # formatted_date = current_date.strftime("%Y/%m/%d")
    # print(formatted_date)
    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 1

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [40]:
link=scrape_link_per_day('2023-11-02')

Scraped 30 links from page 0 url https://www.metrotvnews.com/index/2023-11-02/0
Scraped 30 links from page 1 url https://www.metrotvnews.com/index/2023-11-02/1
Scraped 30 links from page 2 url https://www.metrotvnews.com/index/2023-11-02/2
Scraped 30 links from page 3 url https://www.metrotvnews.com/index/2023-11-02/3
Scraped 30 links from page 4 url https://www.metrotvnews.com/index/2023-11-02/4
Scraped 30 links from page 5 url https://www.metrotvnews.com/index/2023-11-02/5
Scraped 30 links from page 6 url https://www.metrotvnews.com/index/2023-11-02/6
Scraped 23 links from page 7 url https://www.metrotvnews.com/index/2023-11-02/7
Scraped 0 links from page 8 url https://www.metrotvnews.com/index/2023-11-02/8


In [41]:
print(len(link))

233


In [42]:
def scrape_url(url, max_retries=2):
    retries = 0
    while retries < max_retries:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Judul Berita
                title_elem = soup.find('div', {"class": "title"})
                if title_elem:
                    title_text = title_elem.text.strip()
                else:
                    title_text = "Title not found"

                # Tanggal berita
                date_elem = soup.find('div', {"class": "reporter"})
                if date_elem:
                    date_text = date_elem.text.strip()
                    # Define a custom timezone information for WIB
                    def wib_tz(offset):
                        return tz.tzoffset('WIB', offset)

                    # Parse the date string with the custom timezone information
                    date_object = parser.parse(date_text, tzinfos={'WIB': 7*60}, fuzzy=True)

                    # Format the datetime object to the desired format
                    formatted_date = date_object.strftime('%Y-%m-%d')

                else:
                    date_text = "Date not found"

                # Content Berita
                body_elem = soup.find('div', {"class": "read"})

                if body_elem:
                    content_text = body_elem.text
                    content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                    content_text = ' '.join(content_text.split())
                else:
                    content_text = "Content not found"

                return {
                    'title': title_text,
                    'date': formatted_date,
                    'content': content_text,
                    'link': url
                }
            elif response.status_code == 429:
                print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                time.sleep(5)
            else:
                print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL '{url}': {e}")
        except Exception as e:
            print(f"Error processing URL '{url}': {e}")
        retries += 1
        if retries < max_retries:
            print(f"Retrying {url} (Attempt {retries}/{max_retries})")
            time.sleep(5)  # You can adjust the delay as needed
    return None

In [46]:
def scrape_url(url, max_retries=2):
    retries = 0
    while retries < max_retries:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Judul Berita
                title_elem = soup.find('h1')
                if title_elem:
                    title_text = title_elem.text.strip()
                else:
                    title_text = "Title not found"

                # Tanggal berita
                date_elem = soup.find('p', {"class": "pt-20 date"})
                if date_elem:
                    date_text = date_elem.text.strip()
                    parsed_date = datetime.strptime(date_text, 'Media Indonesia • %d %B %Y %H:%M')

                    # Format the date as 'yyyy-mm-dd'
                    formatted_date = parsed_date.strftime('%Y-%m-%d')

                else:
                    date_text = "Date not found"

                # Content Berita
                body_elem = soup.find('div', {"class": "news-text"})

                if body_elem:
                    content_text = body_elem.text
                    content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                    content_text = ' '.join(content_text.split())
                else:
                    content_text = "Content not found"

                return {
                    'title': title_text,
                    'date': formatted_date,
                    'content': content_text,
                    'link': url
                }
            elif response.status_code == 429:
                print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                time.sleep(5)
            else:
                print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL '{url}': {e}")
        except Exception as e:
            print(f"Error processing URL '{url}': {e}")
        retries += 1
        if retries < max_retries:
            print(f"Retrying {url} (Attempt {retries}/{max_retries})")
            time.sleep(5)  # You can adjust the delay as needed
    return None

In [47]:
cek = scrape_url('https://www.metrotvnews.com/read/kewCaRZl-kpu-hapus-debat-khusus-cawapres')

In [48]:
cek

{'title': 'KPU Hapus Debat Khusus Cawapres',
 'date': '2023-12-01',
 'content': 'Jakarta: Komisi Pemilihan Umum (KPU) membuat format baru Debat Pemilihan Presiden (Pilpres) 2024. Format baru tersebut membuat debat khusus antarcawapres ditiadakan karena didampingi calon presiden (capres).Ketua KPU RI Hasyim Asy\'ari menjelaskan kebijakan itu dibuat. Penyelenggara pemilu ingin pemilih melihat sejauh mana kerja sama yang dibangun antara capres dan cawapresnya."Sehingga kemudian publik makin yakin lah teamwork antara capres dan cawapres dalam penampilan di debat," kata Hasyim saat dikonfirmasi, Jumat, 1 Desember 2023.Eks komisioner KPU periode 2027-2022 itu menyampaikan lima rangkaian debat itu diikuti setiap pasangan calon (paslon) Pilpres 2024. Namun, dibagi menjadi debat capres sebanyak tiga kali dan cawapres dua kali. Baca juga: Format Baru, Debat Cawapres Kini Bakal Didampingi CapresPada debat capres, porsi bicara akan lebih banyak diberikan kepada calon RI 1. Sedangkan debat cawapres